In [1]:
import os
import pandas as pd
import re
import numpy as np
import netCDF4 as nc
from PIL import Image
import datetime 
import imageio
import os 
import matplotlib.pyplot as plt
import math
import xarray as xr

# Load Site Data

In [2]:
sites_df = pd.read_csv('~data/site.csv')
sites = [[row['label'], row['EUBWID'], row['lat'], row['long']] for index, row in sites_df.iterrows()]
sites[:10]

[['Ainsdale', 'ukd5300-41300', 53.60792741, -3.063987798],
 ['Allonby', 'ukd1101-46100', 54.76803625, -3.434625292],
 ['Allonby South', 'ukd1101-46000', 54.74396018, -3.456453603],
 ['Amble Links', 'ukc2101-04250', 55.32565184, -1.552400659],
 ['Anderby', 'ukf3102-09400', 53.25977424, 0.325519541],
 ["Anstey's Cove (Torquay)", 'ukk4200-23501', 50.47184615, -3.501089731],
 ['Babbacombe', 'ukk4200-23400', 50.47907733, -3.509409069],
 ['Bamburgh Castle', 'ukc2102-03700', 55.61096248, -1.707903477],
 ['Bantham', 'ukk4305-25700', 50.2789762, -3.878799945],
 ['Beachlands Central', 'ukj3307-16350', 50.78359334, -0.989275631]]

# Load single NetCDF dataset for the lat and lon grid, and snap sites to grid

In [3]:
ds = nc.Dataset("~data/cmems_obs_oc_nws_bgc_tur-spm-chl_nrt_l4-hr-mosaic_P1D-m_1685923516879.nc")
lats = ds['lat'][:].data
lons = ds['lon'][:].data

In [4]:
%%time
sites_grid = []
for site in sites:
    lat_orig = site[2]
    lon_orig = site[3]
    
    j = 25 # this returns 51 x 51 grid with site at the center
    
    lat_grid = lats[np.argmin(np.abs(lats - lat_orig))]
    lon_grid = lons[np.argmin(np.abs(lons - lon_orig))]
    
    lat_grid_left = lats[np.argmin(np.abs(lats - lat_orig))-j]
    lon_grid_left = lons[np.argmin(np.abs(lons - lon_orig))-j]
    
    lat_grid_right = lats[np.argmin(np.abs(lats - lat_orig))+j]
    lon_grid_right = lons[np.argmin(np.abs(lons - lon_orig))+j]
    
    sites_grid.append([site[0], site[1], lat_orig, lon_orig, lat_grid, lon_grid, 
                       lat_grid_left, lon_grid_left, lat_grid_right, lon_grid_right])
    
sites_grid[:5]

CPU times: user 54.7 ms, sys: 2.43 ms, total: 57.1 ms
Wall time: 56.4 ms


[['Ainsdale',
  'ukd5300-41300',
  53.60792741,
  -3.063987798,
  53.60787037037037,
  -3.0634171907756738,
  53.631018518518516,
  -3.1070929419985944,
  53.584722222222226,
  -3.0197414395527513],
 ['Allonby',
  'ukd1101-46100',
  54.76803625,
  -3.434625292,
  54.768055555555556,
  -3.433787561146044,
  54.7912037037037,
  -3.4774633123689647,
  54.74490740740741,
  -3.3901118099231216],
 ['Allonby South',
  'ukd1101-46000',
  54.74396018,
  -3.456453603,
  54.743981481481484,
  -3.4564989517819615,
  54.76712962962963,
  -3.500174703004884,
  54.72083333333333,
  -3.412823200559041],
 ['Amble Links',
  'ukc2101-04250',
  55.32565184,
  -1.552400659,
  55.32546296296296,
  -1.5522361984626052,
  55.34861111111111,
  -1.5959119496855259,
  55.302314814814814,
  -1.5085604472396845],
 ['Anderby',
  'ukf3102-09400',
  53.25977424,
  0.325519541,
  53.25972222222222,
  0.32582110412299947,
  53.28287037037037,
  0.2821453529000788,
  53.23657407407407,
  0.36949685534592014]]

# Loop through multiple sites

## Trial with one site

In [ ]:
print(sites_grid[0])

In [ ]:
%%time
# general
username = "slai"
password = "Capstone2023"
motu_server = "http://nrt.cmems-du.eu/motu-web/Motu"
output_directory = "/Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/copernicus_data"
product_id = "OCEANCOLOUR_NWS_BGC_HR_L4_NRT_009_209-TDS"
dataset_id = "cmems_obs_oc_nws_bgc_tur-spm-chl_nrt_l4-hr-mosaic_P1D-m"

# site info
site = sites_grid[116]
site_name = site[0]
site_id = site[1]
lat_grid_max = site[6]
lat_grid_min = site[8]
lon_grid_max = site[9]
lon_grid_min = site[7]

# query specifications
output_filename = f"{site_id}.nc"
date_min = "2020-01-01 00:00:00"
date_max = "2022-12-31 23:59:59"

# query
query = ('python -m motuclient --motu https://nrt.cmems-du.eu/motu-web/Motu --service-id ' + product_id + ' --product-id ' + dataset_id + 
         f' --longitude-min {lon_grid_min} --longitude-max {lon_grid_max} --latitude-min {lat_grid_min} --latitude-max {lat_grid_max} ' +
         f'--date-min {date_min} --date-max {date_max} ' +
         '--variable CHL --variable SPM --variable TUR ' +
         '--out-dir ' + output_directory + ' --out-name ' + output_filename +
         ' --user ' + username + ' --pwd ' + password)

os.system(query)

In [ ]:
trial_xr = xr.open_dataset(output_directory+'/'+output_filename)
dates_list = [str(pd.Timestamp(trial_xr['time'][:].data[i]).date()) for i in range(237)]

In [ ]:
def site_local_plot_location(ds, time, variable):
    '''
    ds: .nc Dataset for a single location with data in numpy array of shape (11,11)  \
    of date range 01/03/2022 to 31/10/2022 (string)
    
    time: index from 0 to 236 (integer)
    
    variable: 'CHL' or 'TUR' or 'SPM' (string)
    '''
    lats = ds['lat'][:].data
    lons = ds['lon'][:].data

    arr = trial[variable][time,:,:].data

    # Basic plot
    plt.imshow(arr, cmap='viridis', interpolation='nearest')

    # Add highlighted point of Site Location
    plt.scatter(5, 5, s=100, marker='o', facecolors='r')

    # Plot colorbar
    plt.colorbar()

    # Changing Ticks to match latitude and longitude
    new_xticks = [round(i, 3) for i in lons[::5]]
    plt.xticks([0,5,10], new_xticks)
    new_yticks = [round(i, 5) for i in lats[::5]]
    plt.yticks([0,5,10], new_yticks)
    
    # Add Title
    plt.title(f'Time = {dates_list[time]}, {variable}')

In [ ]:
trial = nc.Dataset(output_directory+'/'+output_filename)
site_local_plot_location(trial, 1, 'CHL')

In [ ]:
fig, axs = plt.subplots(nrows=60, ncols=4, figsize=(25, 300))

for i, ax in enumerate(axs.flatten()):
    if i < 237:
        site_local_plot_location(trial, i, 'CHL')
        plt.sca(ax)
        plt.xticks([])
        plt.yticks([])
        ax.set_facecolor('black')
    else:
        ax.axis('off')
        
plt.subplots_adjust(wspace=0.2, hspace=0.6)
plt.show()

In [ ]:
df = trial_xr.to_dataframe()
df['Site'] = site_name
df

# Loop through all sites

## Download .nc files

In [5]:
%%time
# general
username = "slai"
password = "Capstone2023"
output_directory = "/Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/copernicus_data"
product_id = "OCEANCOLOUR_NWS_BGC_HR_L4_NRT_009_209-TDS"
dataset_id = "cmems_obs_oc_nws_bgc_tur-spm-chl_nrt_l4-hr-mosaic_P1D-m"

# Loop through sites_grid (lists of list, with each list corresponding to a site and its corresponding lon/lat markings)
for site in sites_grid[42:]:
    # site info
    site_id = site[1]
    lat_grid_max = site[6]
    lat_grid_min = site[8]
    lon_grid_max = site[9]
    lon_grid_min = site[7]

    # query specifications
    output_filename = f"{site_id}.nc"
    date_min = "2020-01-01 00:00:00"
    date_max = "2022-12-31 23:59:59"

    # query
    query = ('python -m motuclient --motu https://nrt.cmems-du.eu/motu-web/Motu --service-id ' + product_id + ' --product-id ' + dataset_id + 
             f' --longitude-min {lon_grid_min} --longitude-max {lon_grid_max} --latitude-min {lat_grid_min} --latitude-max {lat_grid_max} ' +
             f'--date-min {date_min} --date-max {date_max} ' +
             '--variable CHL --variable SPM --variable TUR ' +
             '--out-dir ' + output_directory + ' --out-name ' + output_filename +
             ' --user ' + username + ' --pwd ' + password)

    os.system(query)

2023-06-06 18:52:09.947 [ INFO] Asynchronous mode set
2023-06-06 18:52:09.947 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-06 18:52:17.894 [ INFO] Requesting file to download (this can take a while)...
2023-06-06 18:52:26.165 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-06 18:52:38.825 [ INFO] Product is not yet available (request in progress)
2023-06-06 18:52:48.831 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-06 18:53:02.883 [ INFO] Product is not yet available (request in progress)
2023-06-06 18:53:12.888 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-06 18:53:29.004 [ INFO] The product is ready for download
2023-06-06 18:53:29.005 [ INFO] Downloading file (this can take a while)...
2023-06-06 18:53:29.174 [ INFO] File type: application/x-netcdf
2023-06-06 18:53:29.177 [ INFO] File size: 30.1 MB (3013289

2023-06-06 18:54:46.708 [ INFO] -  30.1 MB (30.4%)
2023-06-06 18:54:46.719 [ INFO] -  30.1 MB (31.5%)
2023-06-06 18:54:46.743 [ INFO] -  30.1 MB (32.6%)
2023-06-06 18:54:46.752 [ INFO] -  30.1 MB (33.7%)
2023-06-06 18:54:46.780 [ INFO] -  30.1 MB (34.8%)
2023-06-06 18:54:46.790 [ INFO] -  30.1 MB (35.9%)
2023-06-06 18:54:46.823 [ INFO] -  30.1 MB (37.0%)
2023-06-06 18:54:46.828 [ INFO] -  30.1 MB (38.1%)
2023-06-06 18:54:46.854 [ INFO] -  30.1 MB (39.1%)
2023-06-06 18:54:46.860 [ INFO] -  30.1 MB (40.2%)
2023-06-06 18:54:46.884 [ INFO] -  30.1 MB (41.3%)
2023-06-06 18:54:46.892 [ INFO] -  30.1 MB (42.4%)
2023-06-06 18:54:46.914 [ INFO] -  30.1 MB (43.5%)
2023-06-06 18:54:46.926 [ INFO] -  30.1 MB (44.6%)
2023-06-06 18:54:46.944 [ INFO] -  30.1 MB (45.7%)
2023-06-06 18:54:46.956 [ INFO] -  30.1 MB (46.8%)
2023-06-06 18:54:46.970 [ INFO] -  30.1 MB (47.8%)
2023-06-06 18:54:46.985 [ INFO] -  30.1 MB (48.9%)
2023-06-06 18:54:46.994 [ INFO] -  30.1 MB (50.0%)
2023-06-06 18:54:47.013 [ INFO]

2023-06-06 18:56:02.882 [ INFO] -  30.1 MB (77.2%)
2023-06-06 18:56:02.915 [ INFO] -  30.1 MB (78.3%)
2023-06-06 18:56:02.918 [ INFO] -  30.1 MB (79.4%)
2023-06-06 18:56:02.923 [ INFO] -  30.1 MB (80.5%)
2023-06-06 18:56:02.955 [ INFO] -  30.1 MB (81.6%)
2023-06-06 18:56:02.960 [ INFO] -  30.1 MB (82.6%)
2023-06-06 18:56:02.963 [ INFO] -  30.1 MB (83.7%)
2023-06-06 18:56:02.995 [ INFO] -  30.1 MB (84.8%)
2023-06-06 18:56:02.999 [ INFO] -  30.1 MB (85.9%)
2023-06-06 18:56:03.003 [ INFO] -  30.1 MB (87.0%)
2023-06-06 18:56:03.036 [ INFO] -  30.1 MB (88.1%)
2023-06-06 18:56:03.040 [ INFO] -  30.1 MB (89.2%)
2023-06-06 18:56:03.043 [ INFO] -  30.1 MB (90.3%)
2023-06-06 18:56:03.054 [ INFO] -  30.1 MB (91.3%)
2023-06-06 18:56:03.080 [ INFO] -  30.1 MB (92.4%)
2023-06-06 18:56:03.085 [ INFO] -  30.1 MB (93.5%)
2023-06-06 18:56:03.088 [ INFO] -  30.1 MB (94.6%)
2023-06-06 18:56:03.121 [ INFO] -  30.1 MB (95.7%)
2023-06-06 18:56:03.125 [ INFO] -  30.1 MB (96.8%)
2023-06-06 18:56:03.129 [ INFO]

2023-06-06 18:58:27.083 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-06 18:58:35.579 [ INFO] The product is ready for download
2023-06-06 18:58:35.579 [ INFO] Downloading file (this can take a while)...
2023-06-06 18:58:35.729 [ INFO] File type: application/x-netcdf
2023-06-06 18:58:35.730 [ INFO] File size: 30.1 MB (30132892 B)
2023-06-06 18:58:35.730 [ INFO] Downloading file /Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/copernicus_data/ukh3311-11700.nc
2023-06-06 18:58:35.853 [ INFO] -  30.1 MB (1.1%)
2023-06-06 18:58:35.886 [ INFO] -  30.1 MB (2.2%)
2023-06-06 18:58:35.914 [ INFO] -  30.1 MB (3.3%)
2023-06-06 18:58:35.920 [ INFO] -  30.1 MB (4.3%)
2023-06-06 18:58:35.925 [ INFO] -  30.1 MB (5.4%)
2023-06-06 18:58:35.948 [ INFO] -  30.1 MB (6.5%)
2023-06-06 18:58:35.961 [ INFO] -  30.1 MB (7.6%)
2023-06-06 18:58:35.964 [ INFO] -  30.1 MB (8.7%)
2023-06-06 18:58:35.980 [ INFO] -  30.1 MB (9.8%)
20

2023-06-06 18:59:54.265 [ INFO] -  30.1 MB (34.8%)
2023-06-06 18:59:54.282 [ INFO] -  30.1 MB (35.9%)
2023-06-06 18:59:54.305 [ INFO] -  30.1 MB (37.0%)
2023-06-06 18:59:54.333 [ INFO] -  30.1 MB (38.1%)
2023-06-06 18:59:54.359 [ INFO] -  30.1 MB (39.1%)
2023-06-06 18:59:54.382 [ INFO] -  30.1 MB (40.2%)
2023-06-06 18:59:54.399 [ INFO] -  30.1 MB (41.3%)
2023-06-06 18:59:54.405 [ INFO] -  30.1 MB (42.4%)
2023-06-06 18:59:54.415 [ INFO] -  30.1 MB (43.5%)
2023-06-06 18:59:54.428 [ INFO] -  30.1 MB (44.6%)
2023-06-06 18:59:54.435 [ INFO] -  30.1 MB (45.7%)
2023-06-06 18:59:54.449 [ INFO] -  30.1 MB (46.8%)
2023-06-06 18:59:54.460 [ INFO] -  30.1 MB (47.8%)
2023-06-06 18:59:54.475 [ INFO] -  30.1 MB (48.9%)
2023-06-06 18:59:54.484 [ INFO] -  30.1 MB (50.0%)
2023-06-06 18:59:54.507 [ INFO] -  30.1 MB (51.1%)
2023-06-06 18:59:54.536 [ INFO] -  30.1 MB (52.2%)
2023-06-06 18:59:54.537 [ INFO] -  30.1 MB (53.3%)
2023-06-06 18:59:54.538 [ INFO] -  30.1 MB (54.4%)
2023-06-06 18:59:54.539 [ INFO]

...

2023-06-07 03:24:58.662 [ INFO] -  30.1 MB (62.0%)
2023-06-07 03:24:58.671 [ INFO] -  30.1 MB (63.1%)
2023-06-07 03:24:58.675 [ INFO] -  30.1 MB (64.2%)
2023-06-07 03:24:58.688 [ INFO] -  30.1 MB (65.2%)
2023-06-07 03:24:58.704 [ INFO] -  30.1 MB (66.3%)
2023-06-07 03:24:58.707 [ INFO] -  30.1 MB (67.4%)
2023-06-07 03:24:58.712 [ INFO] -  30.1 MB (68.5%)
2023-06-07 03:24:58.737 [ INFO] -  30.1 MB (69.6%)
2023-06-07 03:24:58.740 [ INFO] -  30.1 MB (70.7%)
2023-06-07 03:24:58.745 [ INFO] -  30.1 MB (71.8%)
2023-06-07 03:24:58.749 [ INFO] -  30.1 MB (72.9%)
2023-06-07 03:24:58.775 [ INFO] -  30.1 MB (73.9%)
2023-06-07 03:24:58.779 [ INFO] -  30.1 MB (75.0%)
2023-06-07 03:24:58.782 [ INFO] -  30.1 MB (76.1%)
2023-06-07 03:24:58.813 [ INFO] -  30.1 MB (77.2%)
2023-06-07 03:24:58.815 [ INFO] -  30.1 MB (78.3%)
2023-06-07 03:24:58.818 [ INFO] -  30.1 MB (79.4%)
2023-06-07 03:24:58.846 [ INFO] -  30.1 MB (80.5%)
2023-06-07 03:24:58.849 [ INFO] -  30.1 MB (81.6%)
2023-06-07 03:24:58.854 [ INFO]

2023-06-07 03:26:01.510 [ INFO] Asynchronous mode set
2023-06-07 03:26:01.510 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-07 03:26:02.120 [ INFO] Requesting file to download (this can take a while)...
2023-06-07 03:26:07.624 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-07 03:26:13.727 [ INFO] Product is not yet available (request in progress)
2023-06-07 03:26:23.738 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-07 03:26:29.930 [ INFO] Product is not yet available (request in progress)
2023-06-07 03:26:39.937 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-07 03:26:46.088 [ INFO] Product is not yet available (request in progress)
2023-06-07 03:26:56.091 [ INFO] Authenticating user slai for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-07 03:27:02.170 [ INFO] The product is ready for download
2023-06-0

2023-06-07 03:28:04.612 [ INFO] -  30.1 MB (18.5%)
2023-06-07 03:28:04.639 [ INFO] -  30.1 MB (19.6%)
2023-06-07 03:28:04.643 [ INFO] -  30.1 MB (20.7%)
2023-06-07 03:28:04.649 [ INFO] -  30.1 MB (21.7%)
2023-06-07 03:28:04.672 [ INFO] -  30.1 MB (22.8%)
2023-06-07 03:28:04.680 [ INFO] -  30.1 MB (23.9%)
2023-06-07 03:28:04.685 [ INFO] -  30.1 MB (25.0%)
2023-06-07 03:28:04.692 [ INFO] -  30.1 MB (26.1%)
2023-06-07 03:28:04.723 [ INFO] -  30.1 MB (27.2%)
2023-06-07 03:28:04.728 [ INFO] -  30.1 MB (28.3%)
2023-06-07 03:28:04.730 [ INFO] -  30.1 MB (29.4%)
2023-06-07 03:28:04.761 [ INFO] -  30.1 MB (30.4%)
2023-06-07 03:28:04.766 [ INFO] -  30.1 MB (31.5%)
2023-06-07 03:28:04.771 [ INFO] -  30.1 MB (32.6%)
2023-06-07 03:28:04.801 [ INFO] -  30.1 MB (33.7%)
2023-06-07 03:28:04.806 [ INFO] -  30.1 MB (34.8%)
2023-06-07 03:28:04.811 [ INFO] -  30.1 MB (35.9%)
2023-06-07 03:28:04.843 [ INFO] -  30.1 MB (37.0%)
2023-06-07 03:28:04.846 [ INFO] -  30.1 MB (38.1%)
2023-06-07 03:28:04.848 [ INFO]

2023-06-07 03:29:06.973 [ INFO] -  30.1 MB (59.8%)
2023-06-07 03:29:06.978 [ INFO] -  30.1 MB (60.9%)
2023-06-07 03:29:06.981 [ INFO] -  30.1 MB (62.0%)
2023-06-07 03:29:07.005 [ INFO] -  30.1 MB (63.1%)
2023-06-07 03:29:07.010 [ INFO] -  30.1 MB (64.2%)
2023-06-07 03:29:07.013 [ INFO] -  30.1 MB (65.2%)
2023-06-07 03:29:07.017 [ INFO] -  30.1 MB (66.3%)
2023-06-07 03:29:07.047 [ INFO] -  30.1 MB (67.4%)
2023-06-07 03:29:07.051 [ INFO] -  30.1 MB (68.5%)
2023-06-07 03:29:07.056 [ INFO] -  30.1 MB (69.6%)
2023-06-07 03:29:07.081 [ INFO] -  30.1 MB (70.7%)
2023-06-07 03:29:07.085 [ INFO] -  30.1 MB (71.8%)
2023-06-07 03:29:07.089 [ INFO] -  30.1 MB (72.9%)
2023-06-07 03:29:07.093 [ INFO] -  30.1 MB (73.9%)
2023-06-07 03:29:07.117 [ INFO] -  30.1 MB (75.0%)
2023-06-07 03:29:07.121 [ INFO] -  30.1 MB (76.1%)
2023-06-07 03:29:07.126 [ INFO] -  30.1 MB (77.2%)
2023-06-07 03:29:07.153 [ INFO] -  30.1 MB (78.3%)
2023-06-07 03:29:07.157 [ INFO] -  30.1 MB (79.4%)
2023-06-07 03:29:07.161 [ INFO]

2023-06-07 03:30:09.533 [ INFO] -  30.1 MB (100.0%)
2023-06-07 03:30:09.533 [ INFO] Processing  time : 0:01:00.680697
2023-06-07 03:30:09.534 [ INFO] Downloading time : 0:00:01.181207
2023-06-07 03:30:09.534 [ INFO] Total time       : 0:01:01.861904
2023-06-07 03:30:09.534 [ INFO] Download rate    : 22.3 MB/s
2023-06-07 03:30:09.537 [ INFO] Done
CPU times: user 21.3 s, sys: 5.44 s, total: 26.7 s
Wall time: 8h 37min 59s


## Convert to CSV and merge

In [6]:
import xarray as xr
import pandas as pd
import glob

In [7]:
nc_files = glob.glob('/Users/yuhanwang/Desktop/CourseMaterial/Capstone/capstone-project-sewage-pollution/~data/copernicus_data/*.nc')
ds_list = [xr.open_dataset(file) for file in nc_files]

In [9]:
with open("sites_data_51x51.csv", 'w') as f:
    for file in nc_files:

        ds = xr.open_dataset(file)
        df = ds.to_dataframe()

        site = file.split('/')[-1].replace(".nc", "")
        df['site'] = site

        df = df.dropna(subset=['TUR', 'SPM', 'CHL'], how='all')

        if file == nc_files[0]:
            df.to_csv(f)
        else:
            df.to_csv(f, mode='a', header=False)

        ds.close()

In [ ]:
df.groupby(['time', 'lon'], as_index=False).size()